<a href="https://colab.research.google.com/github/LuizNunes2020/Laboratorios_ME315/blob/main/Lab6_SQLite_Parte_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Instalação r2u

shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
shell_call("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
shell_call("rm add_cranapt_jammy.sh")

## Preparando acesso ao Google Drive
loadPackages = function(pkgs){
  myrequire = function(...){
    suppressWarnings(suppressMessages(suppressPackageStartupMessages(require(...))))
  }
  ok = sapply(pkgs, require, character.only=TRUE, quietly=TRUE)
  if (!all(ok)){
    message("There are missing packages: ", paste(pkgs[!ok], collapse=", "))
    install.packages(pkgs[!ok])
    loadPackages(pkgs[!ok])
  }
}

loadPackages(c("googledrive", "httpuv", "R.utils", "httr"))

my_check = function() TRUE
reassignInPackage("is_interactive", pkgName="httr", my_check)
options(rlang_interactive=TRUE)

52 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
-----BEGIN PGP PUBLIC KEY BLOCK-----

...
I5rGSOk715L9
=VuA5
-----END PGP PUBLIC KEY BLOCK-----
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit

Tracing function "install.packages" in package "utils"

There are missing packages: R.utils


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save



Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




In [ ]:
## O comando abaixo permite acesso ao Google Drive
drive_auth(use_oob=TRUE, cache=FALSE)

1. Baixe o arquivo disco.db e armazene na variável path o caminho completo (pasta) na qual o arquivo foi gravado. Utilize o comando file.path() para combinar a variável path com o nome do arquivo (disco.db) e obter o nome do arquivo com seu respectivo caminho. Armazene este resultado na variável fname.



In [ ]:
path = "https://me315-unicamp.github.io/dados/disco.db"
fname = file.path(path,"disco.db")
download.file(path,"disco.db")

In [ ]:
file.exists("disco.db")

[1] TRUE

2. Utilizando o pacote RSQLite, conecte-se ao arquivo de banco de dados. Armazene a conexão na variável conn.

In [ ]:
install.packages("RSQLite")
library(RSQLite)
library(dplyr)
library(tidyverse)
conn = dbConnect(SQLite(), "disco.db")

3. Liste as tabelas existentes no banco de dados.

In [ ]:
length(dbListTables(conn))
tabs = dbListTables(conn)

[1] 13

[1] "albums"          "artists"         "customers"       "employees"      
 [5] "genres"          "invoice_items"   "invoices"        "media_types"    
 [9] "playlist_track"  "playlists"       "sqlite_sequence" "sqlite_stat1"   
[13] "tracks"

In [ ]:
4. Identifique os nomes de todas as colunas existentes na tabela customers.

In [ ]:
dbListFields(conn,"customers")

[1] "CustomerId"   "FirstName"    "LastName"     "Company"      "Address"     
 [6] "City"         "State"        "Country"      "PostalCode"   "Phone"       
[11] "Fax"          "Email"        "SupportRepId"

5. Utilizando apenas SQLite, com o apoio do comando dbGetQuery, identifique quantos clientes estão atualmente cadastrados neste banco de dados.

In [ ]:
(dbGetQuery(conn,"SELECT COUNT(CustomerId) FROM customers"))

COUNT(CustomerId)
<int>
59


6. Utilizando apenas SQLite, identifique o número de países diferentes em que moram os clientes encontrados acima.

In [ ]:
(dbGetQuery(conn,"SELECT COUNT(DISTINCT Country) FROM customers ORDER BY CustomerID"))

COUNT(DISTINCT Country)
<int>
24


7. Utilizando apenas SQLite, quantos clientes existem por país? A tabela resultante deve conter o nome do país e a respectiva contagem, além de ser ordenada de maneira decrescente pela referida contagem.

In [ ]:
(dbGetQuery(conn,"SELECT Country,COUNT(CustomerID) AS n FROM customers GROUP BY Country ORDER BY n DESC"))

Country,n
<chr>,<int>
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2


8. Quais são os 5 países com mais clientes registrados? Use apenas SQLite.

In [ ]:
(dbGetQuery(conn,"SELECT Country,COUNT(CustomerID) AS n FROM customers GROUP BY Country ORDER BY n DESC LIMIT 5"))

Country,n
<chr>,<int>
USA,13
Canada,8
France,5
Brazil,5
Germany,4


9. Quais são os países registrados que possuem apenas 6 letras no nome?

In [ ]:
(dbGetQuery(conn,"SELECT DISTINCT Country FROM customers WHERE Country GLOB '??????'"))

Country
<chr>
Brazil
Canada
Norway
France
Poland
Sweden


In [ ]:
(dbGetQuery(conn,"SELECT DISTINCT country FROM customers WHERE LENGTH(Country) = 6"))

Country
<chr>
Brazil
Canada
Norway
France
Poland
Sweden


10. Quais foram as músicas compradas por clientes brasileiros?

In [ ]:
(dbGetQuery(conn,"SELECT DISTINCT Name FROM tracks INNER JOIN invoice_items ON tracks.trackid = invoice_items.trackid INNER JOIN invoices ON invoice_items.invoiceid = invoices.invoiceid INNER JOIN customers ON invoices.customerid = customers.customerid WHERE Country = 'Brazil'"))

Name
<chr>
Experiment In Terra
Take the Celestra
Shout It Out Loud
Calling Dr. Love
Strutter
Cold Gin
Think About You
Rocket Queen
Don't Cry (Original)


11. Desconecte do banco de dados

In [ ]:
dbDisconnect(conn)